In [11]:
import os
import sys

# Add the 'src' directory to the Python path so we can import 'meal_planner'
# This assumes you started Jupyter Lab from the project root ('smart-meal-planner')
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now import the function from our module
from meal_planner.user_input import load_user_profile

print("Import successful!")

Import successful!


In [23]:
# Define the path to the example user profile, relative to the project root
profile_file_path = os.path.join('..', 'data', 'examples', 'user_profile_1.json')
# Convert to absolute path for clarity in printing
abs_profile_path = os.path.abspath(profile_file_path)
print(f"Attempting to load profile from: {abs_profile_path}")

# Call the function to load the profile
user_profile = load_user_profile(abs_profile_path) # Use absolute path here

# Check if loading was successful and print the result
if user_profile:
    print("\n--- Loaded User Profile ---")
    print(user_profile)
    print(f"\nType of loaded data: {type(user_profile)}")
else:
    print("\nFailed to load user profile.")

Attempting to load profile from: c:\Users\bhara\Desktop\Meal_Planner\data\examples\user_profile_1.json
Successfully loaded user profile from: c:\Users\bhara\Desktop\Meal_Planner\data\examples\user_profile_1.json

--- Loaded User Profile ---
{'user_id': 'user123', 'fitness_goal': 'weight loss', 'dietary_preferences': ['vegetarian'], 'allergies_or_avoidances': ['nuts', 'shellfish'], 'taste_preferences': ['likes spicy', 'prefers savory breakfast'], 'schedule_constraints': ['dinner out Wednesday', 'needs quick lunches (under 30 mins prep+cook)'], 'specific_requests': ['Include pasta at least twice', 'No mushrooms']}

Type of loaded data: <class 'dict'>


In [22]:
os.path.exists(r"C:\Users\bhara\Desktop\Meal_Planner\data\examples\user_profile_1.json")

True

In [18]:
abs_profile_path

'c:\\Users\\bhara\\Desktop\\Meal_Planner\\data\\examples\\user_profile_1.json'

In [24]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import json # Import json for parsing potential JSON output


In [25]:
# Load environment variables from .env file
load_dotenv()

# Configure the generative AI client
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not found in .env file or environment variables.")

genai.configure(api_key=api_key)

In [26]:

# --- Configuration ---
# Choose the model appropriate for text generation and potentially JSON output
# 'gemini-pro' is a good starting point.
# Models supporting explicit JSON mode might be named differently (e.g., check Gemini docs)
MODEL_NAME = "gemini-1.5-pro-latest" # Or "gemini-pro" if 1.5 isn't available/needed yet

# Define generation configuration for potentially forcing JSON output
# Note: Explicit JSON mode might require specific model versions or settings.
# Refer to Google AI documentation for the latest on enforcing JSON.
# This is a general approach; specific JSON mode might use different parameters.
GENERATION_CONFIG_JSON = {
  "temperature": 0.5, # Adjust creativity (lower is more deterministic)
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 2048, # Adjust as needed for plan length
  "response_mime_type": "application/json", # Request JSON output
}

# --- Client Function ---

def generate_text_with_retry(prompt_text, generation_config=None, safety_settings=None, retries=2):
    """
    Calls the Gemini API to generate text based on a prompt, with retries.

    Args:
        prompt_text (str): The input prompt for the model.
        generation_config (dict, optional): Configuration for generation (temp, tokens, etc.).
                                            Defaults to a basic config if None.
        safety_settings (list, optional): Configuration for safety filters. Defaults to None.
        retries (int): Number of times to retry on specific errors.

    Returns:
        str: The generated text content from the model, or None if generation fails after retries.
    """
    if generation_config is None:
        # Default config if none provided (might not force JSON here)
        generation_config = {
            "temperature": 0.7,
            "top_p": 0.95,
            "top_k": 40,
            "max_output_tokens": 2048,
        }

    model = genai.GenerativeModel(
        model_name=MODEL_NAME,
        generation_config=generation_config,
        safety_settings=safety_settings
        # system_instruction="You are a helpful meal planning assistant." # Optional: Set system-level instructions
    )

    attempt = 0
    while attempt <= retries:
        try:
            print(f"\n--- Calling Gemini API (Attempt {attempt + 1}/{retries + 1}) ---")
            response = model.generate_content(prompt_text)
            print("--- Gemini API Call Successful ---")

            # Basic check if response has text content
            if response.parts:
                 # Accessing the text directly, assuming it's the first part
                 # If using JSON mode, this should ideally be the JSON string
                generated_content = response.text
                # print(f"Raw Response Text:\n{generated_content[:500]}...") # Optional: Print start of raw response
                return generated_content
            else:
                 # Handle cases where the response might be blocked or empty
                 print("Warning: Response received but contains no usable parts.")
                 # Check for blocking reasons if available
                 if response.prompt_feedback and response.prompt_feedback.block_reason:
                     print(f"Prompt blocked due to: {response.prompt_feedback.block_reason}")
                 # Check candidates for finish reasons
                 if response.candidates and response.candidates[0].finish_reason != 'STOP':
                     print(f"Generation stopped due to: {response.candidates[0].finish_reason}")
                 return None # Indicate failure or empty response

        except Exception as e:
            # Catching a broad range of potential API errors
            print(f"Error during Gemini API call (Attempt {attempt + 1}): {e}")
            attempt += 1
            if attempt > retries:
                print("Max retries reached. Generation failed.")
                return None
            print("Retrying...")
            # Consider adding a small delay here before retrying: time.sleep(1)

    return None # Should not be reached if loop logic is correct, but acts as fallback


In [27]:

# --- Helper to Parse JSON ---
def parse_json_output(text_output):
    """
    Attempts to parse a string assumed to contain JSON into a Python dictionary.

    Args:
        text_output (str): The string output from the LLM.

    Returns:
        dict: The parsed dictionary, or None if parsing fails or input is None.
    """
    if not text_output:
        print("Error: No text output received from LLM to parse.")
        return None

    try:
        # Sometimes the LLM might wrap the JSON in markdown ```json ... ```
        if text_output.strip().startswith("```json"):
            print("Detected JSON wrapped in markdown, attempting to extract.")
            # Extract content between the first ```json and the last ```
            json_str = text_output.split("```json", 1)[1].rsplit("```", 1)[0].strip()
        elif text_output.strip().startswith("{") and text_output.strip().endswith("}"):
             # Assume it's already a JSON string if it starts/ends with braces
             json_str = text_output.strip()
        else:
            # If it doesn't look like JSON, maybe the LLM failed the instruction
            print("Warning: Output doesn't look like JSON. Attempting direct parse anyway.")
            json_str = text_output.strip() # Try parsing directly just in case

        parsed_json = json.loads(json_str)
        print("Successfully parsed JSON output.")
        return parsed_json
    except json.JSONDecodeError as e:
        print(f"Error: Failed to decode JSON from LLM output. Error: {e}")
        print(f"Problematic Text (first 500 chars):\n{text_output[:500]}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during JSON parsing: {e}")
        return None


In [28]:

# Example Usage (Optional - for testing this file directly)
if __name__ == '__main__':
    print("Testing LLM Client...")
    # Simple test prompt - NOT asking for JSON here, just testing connection
    test_prompt = "Explain the concept of a virtual environment in Python in one sentence."
    print(f"Sending test prompt: '{test_prompt}'")
    result = generate_text_with_retry(test_prompt)

    if result:
        print("\n--- Test Response ---")
        print(result)
    else:
        print("\n--- Test Failed ---")

    # Test JSON parsing helper
    print("\n--- Testing JSON Parsing ---")
    good_json_string = '```json\n{\n  "day_1": {\n    "breakfast": "Oatmeal",\n    "lunch": "Salad",\n    "dinner": "Chicken"\n  }\n}\n```'
    bad_json_string = 'Here is the plan: { "day_1": "bad json" '
    none_input = None

    print("\nTesting good JSON string:")
    parsed = parse_json_output(good_json_string)
    if parsed: print(parsed)

    print("\nTesting bad JSON string:")
    parsed = parse_json_output(bad_json_string)
    if parsed: print(parsed) # Should fail

    print("\nTesting None input:")
    parsed = parse_json_output(none_input)
    if parsed: print(parsed) # Should fail

Testing LLM Client...
Sending test prompt: 'Explain the concept of a virtual environment in Python in one sentence.'

--- Calling Gemini API (Attempt 1/3) ---
--- Gemini API Call Successful ---

--- Test Response ---
A virtual environment is an isolated Python environment that allows you to install packages and dependencies for a specific project without affecting your global Python installation or other projects.


--- Testing JSON Parsing ---

Testing good JSON string:
Detected JSON wrapped in markdown, attempting to extract.
Successfully parsed JSON output.
{'day_1': {'breakfast': 'Oatmeal', 'lunch': 'Salad', 'dinner': 'Chicken'}}

Testing bad JSON string:
Error: Failed to decode JSON from LLM output. Error: Expecting value: line 1 column 1 (char 0)
Problematic Text (first 500 chars):
Here is the plan: { "day_1": "bad json" 

Testing None input:
Error: No text output received from LLM to parse.


In [30]:
import os
import yaml # For loading the prompt from YAML
import json # For potentially validating/handling JSON

# Import functions from our other modules within the same package
from .llm_client import generate_text_with_retry, parse_json_output, GENERATION_CONFIG_JSON
from .user_input import load_user_profile


ImportError: attempted relative import with no known parent package